In [21]:
# import librariess
import pandas as pd
import numpy as np

In [22]:
# load in the metro dataset
data_dir_metro = "https://raw.githubusercontent.com/chloefeehan/GSB544-Project/refs/heads/main/data/metro_monthly.csv"
metro = pd.read_csv(data_dir_metro)

# remove commas in numerical columns
metro["total_ridership"] = metro["total_ridership"].str.replace(",", "").astype(int)
metro["total_miles"] = metro["total_miles"].str.replace(",", "").astype(int)

# Remove the day_count column
metro = metro.drop(columns=["day_count"])

# create emission column
metro['emission'] = np.where(
    metro['transportation_type'] == 'rail',
    metro['total_miles'] * 0.365,
    metro['total_miles'] * 0.643
)

In [23]:
# load in the gas dataset
data_dir_gas = "https://raw.githubusercontent.com/chloefeehan/GSB544-Project/refs/heads/main/data/gas.csv"
gas = pd.read_csv(data_dir_gas, skiprows = 4)

#split date column into month, date, and year columns
gas[["month", "date", "year"]] = gas["Week of"].str.split("/", expand=True).astype(int)

#filter the data by years 2017 - 2023
filtered_gas = gas[(gas["year"] >= 2017) & (gas["year"] <= 2023)]

# get the average monthly gas for each year
monthly_avg_gas = (
    filtered_gas.groupby(["year", "month"])["Weekly Los Angeles Regular All Formulations Retail Gasoline Prices Dollars per Gallon"]
    .mean()
    .reset_index(name="avg_price")
)

In [24]:
# merge the two datasets
data_clean = metro.merge(monthly_avg_gas, on = ["year", "month"], how = "left")

# create dummy variables for bus and rail
data_clean["bus"] = np.where(data_clean["transportation_type"] == "all bus", 1, 0)
data_clean["rail"] = np.where(data_clean["transportation_type"] == "rail", 1, 0)

In [25]:
# write cleaned data into a csv
data_clean.to_csv("data_clean.csv")